# Kaggle Titanic
## Logistic Regression with Python


For this lecture we will be working with the [Titanic Data Set from Kaggle](https://www.kaggle.com/c/titanic). This is a very famous dataset.


# Step - 1 : Frame The Problem

The sinking of the RMS Titanic is one of the most infamous shipwrecks in history.  On April 15, 1912, during her maiden voyage, the Titanic sank after colliding with an iceberg, killing 1502 out of 2224 passengers and crew. This sensational tragedy shocked the international community and led to better safety regulations for ships.

One of the reasons that the shipwreck led to such loss of life was that there were not enough lifeboats for the passengers and crew. Although there was some element of luck involved in surviving the sinking, some groups of people were more likely to survive than others, such as women, children, and the upper-class.

In this challenge, we ask you to complete the analysis of what sorts of people were likely to survive. In particular, we ask you to apply the tools of machine learning to predict which passengers survived the tragedy.



# Step - 2 : Obtain the Data

## Import Libraries

In [70]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

Pandas provides two important data types with in built functions to be able to provide extensive capability to handle the data.The datatypes include Series and DataFrames.

Pandas provides ways to read or get the data from various sources like read_csv,read_excel,read_html etc.The data is read and stored in the form of DataFrames.

In [71]:
# FOR GOOGLE COLAB, UNCOMMENT LINES BELOW

# !wget -q https://www.dropbox.com/s/8grgwn4b6y25frw/titanic.csv
# !ls -l
# data = pd.read_csv('titanic.csv')

In [72]:
# FOR JUPYTER NOTEBOOK, UNCOMMENT LINES BELOW

data = pd.read_csv('Titanic Data-Train.csv')

In [73]:
data.set_index('PassengerId', inplace=True)

In [74]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 11 columns):
Survived    891 non-null int64
Pclass      891 non-null int64
Name        891 non-null object
Sex         891 non-null object
Age         714 non-null float64
SibSp       891 non-null int64
Parch       891 non-null int64
Ticket      891 non-null object
Fare        891 non-null float64
Cabin       204 non-null object
Embarked    889 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 83.5+ KB


In [75]:
data.describe()

,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


# Step - 3 : Analyse the Data

#### What do you observe from the above charts?

# Step - 4 : Feature Engineering

## Feature Engineering

We want to fill the missing values of the age in the dataset with the average age value for each of the classes. This is called data imputation.

In [76]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 11 columns):
Survived    891 non-null int64
Pclass      891 non-null int64
Name        891 non-null object
Sex         891 non-null object
Age         714 non-null float64
SibSp       891 non-null int64
Parch       891 non-null int64
Ticket      891 non-null object
Fare        891 non-null float64
Cabin       204 non-null object
Embarked    889 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 83.5+ KB


In [77]:
# BL NOTE - SKIP THIS AND USE THE PANDAS CODE I CREATED BELOW
# def impute_age(cols):
#     Age = cols[0]
#     Pclass = cols[1]
    
#     if pd.isnull(Age):
#         # Class-1
#         if Pclass == 1:
#             return 37
#         # Class-2 
#         elif Pclass == 2:
#             return 29
#         # Class-3
#         else:
#             return 24

#     else:
#         return Age

Applying the function.

In [78]:
# data['Age'] = data[['Age','Pclass']].apply(impute_age,axis=1)

In [79]:
# BL NOTE - USE MY CODE TO GET THE MEAN VALUE FOR EACH CLASS INSTEAD OF HARD CODED NUMBERS
mean_age_Pclass1 = int(data.loc[data['Pclass']==1]['Age'].mean())
mean_age_Pclass2 = int(data.loc[data['Pclass']==2]['Age'].mean())
mean_age_Pclass3 = int(data.loc[data['Pclass']==3]['Age'].mean())

data.loc[(data['Age'].isnull() ) & (data['Pclass']==1), 'Age'] = mean_age_Pclass1
data.loc[(data['Age'].isnull() ) & (data['Pclass']==2), 'Age'] = mean_age_Pclass2
data.loc[(data['Age'].isnull() ) & (data['Pclass']==3), 'Age'] = mean_age_Pclass3

Now let's visualize the missing values.

The Age column is imputed sucessfully.

Let's drop the Cabin column and the row in the Embarked that is NaN.

In [80]:
data.drop('Cabin', axis = 1,inplace=True)

In [81]:
data.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
PassengerId,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S


In [82]:
data.isnull().sum()

Survived    0
Pclass      0
Name        0
Sex         0
Age         0
SibSp       0
Parch       0
Ticket      0
Fare        0
Embarked    2
dtype: int64

In [83]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 10 columns):
Survived    891 non-null int64
Pclass      891 non-null int64
Name        891 non-null object
Sex         891 non-null object
Age         891 non-null float64
SibSp       891 non-null int64
Parch       891 non-null int64
Ticket      891 non-null object
Fare        891 non-null float64
Embarked    889 non-null object
dtypes: float64(2), int64(4), object(4)
memory usage: 76.6+ KB


In [84]:
data.dropna(inplace = True)

In [85]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 889 entries, 1 to 891
Data columns (total 10 columns):
Survived    889 non-null int64
Pclass      889 non-null int64
Name        889 non-null object
Sex         889 non-null object
Age         889 non-null float64
SibSp       889 non-null int64
Parch       889 non-null int64
Ticket      889 non-null object
Fare        889 non-null float64
Embarked    889 non-null object
dtypes: float64(2), int64(4), object(4)
memory usage: 76.4+ KB


## Converting Categorical Features 

We'll need to convert categorical features to dummy variables using pandas! Otherwise our machine learning algorithm won't be able to directly take in those features as inputs.

In [86]:
sex_dummies = pd.get_dummies(data['Sex'],drop_first=1)
embark_dummies = pd.get_dummies(data['Embarked'],drop_first=1)
sex_dummies.head()

,male
PassengerId,
1,1
2,0
3,0
4,0
5,1


In [87]:
embark_dummies.head()

,Q,S
PassengerId,,
1,0,1
2,0,0
3,0,1
4,0,1
5,0,1


In [88]:
data.drop(['Sex','Embarked','Name','Ticket'],axis=1,inplace=True)
data.head()

,Survived,Pclass,Age,SibSp,Parch,Fare
PassengerId,,,,,,
1,0,3,22.0,1,0,7.2500
2,1,1,38.0,1,0,71.2833
3,1,3,26.0,0,0,7.9250
4,1,1,35.0,1,0,53.1000
5,0,3,35.0,0,0,8.0500


In [89]:
data = pd.concat([data,sex_dummies,embark_dummies],axis=1)

In [90]:
data.head()

,Survived,Pclass,Age,SibSp,Parch,Fare,male,Q,S
PassengerId,,,,,,,,,
1,0,3,22.0,1,0,7.2500,1,0,1
2,1,1,38.0,1,0,71.2833,0,0,0
3,1,3,26.0,0,0,7.9250,0,0,1
4,1,1,35.0,1,0,53.1000,0,0,1
5,0,3,35.0,0,0,8.0500,1,0,1


In [91]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 889 entries, 1 to 891
Data columns (total 9 columns):
Survived    889 non-null int64
Pclass      889 non-null int64
Age         889 non-null float64
SibSp       889 non-null int64
Parch       889 non-null int64
Fare        889 non-null float64
male        889 non-null uint8
Q           889 non-null uint8
S           889 non-null uint8
dtypes: float64(2), int64(4), uint8(3)
memory usage: 51.2 KB


In [92]:
data.describe()

,Survived,Pclass,Age,SibSp,Parch,Fare,male,Q,S
count,889.000000,889.000000,889.000000,889.000000,889.000000,889.000000,889.000000,889.000000,889.000000
mean,0.382452,2.311586,29.206040,0.524184,0.382452,32.096681,0.649044,0.086614,0.724409
std,0.486260,0.834700,13.177747,1.103705,0.806761,49.697504,0.477538,0.281427,0.447063
min,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,22.000000,0.000000,0.000000,7.895800,0.000000,0.000000,0.000000
50%,0.000000,3.000000,26.000000,0.000000,0.000000,14.454200,1.000000,0.000000,1.000000
75%,1.000000,3.000000,36.500000,1.000000,0.000000,31.000000,1.000000,0.000000,1.000000
max,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200,1.000000,1.000000,1.000000


# Step - 5 : Model Selection

In [93]:
Target = 'Survived'
X = data.drop(Target,axis=1)
y = data[Target]

In [94]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score,accuracy_score, f1_score

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)


In [95]:
X_train.describe()

,Pclass,Age,SibSp,Parch,Fare,male,Q,S
count,711.000000,711.000000,711.000000,711.000000,711.000000,711.000000,711.000000,711.000000
mean,2.320675,29.040211,0.527426,0.379747,32.102818,0.649789,0.085795,0.720113
std,0.829802,13.350864,1.093653,0.820055,51.872981,0.477372,0.280258,0.449260
min,1.000000,0.420000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,22.000000,0.000000,0.000000,7.910400,0.000000,0.000000,0.000000
50%,3.000000,25.000000,0.000000,0.000000,14.454200,1.000000,0.000000,1.000000
75%,3.000000,36.000000,1.000000,0.000000,30.500000,1.000000,0.000000,1.000000
max,3.000000,80.000000,8.000000,6.000000,512.329200,1.000000,1.000000,1.000000


**Standard Scaling**: Standardize features by removing the mean and scaling to unit variance  
Similar to normalization but changes the mean to be 0 and std dev = 1 (create bell curve for all data)  
[Standard Scaling SK Learn](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html)

In [96]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
sc.fit(X_train)
X_train = sc.transform(X_train)

In [97]:
X_train = pd.DataFrame(X_train)
X_train.describe()

,0,1,2,3,4,5,6,7
count,7.110000e+02,7.110000e+02,7.110000e+02,7.110000e+02,7.110000e+02,7.110000e+02,7.110000e+02,7.110000e+02
mean,-6.870578e-17,8.978597e-17,4.591576e-16,1.826949e-16,5.664324e-17,1.052448e-16,2.005740e-16,-2.492146e-16
std,1.000704e+00,1.000704e+00,1.000704e+00,1.000704e+00,1.000704e+00,1.000704e+00,1.000704e+00,1.000704e+00
min,-1.592674e+00,-2.145206e+00,-4.826003e-01,-4.634007e-01,-6.193093e-01,-1.362139e+00,-3.063432e-01,-1.604015e+00
25%,-3.867196e-01,-5.276937e-01,-4.826003e-01,-4.634007e-01,-4.667063e-01,-1.362139e+00,-3.063432e-01,-1.604015e+00
50%,8.192350e-01,-3.028310e-01,-4.826003e-01,-4.634007e-01,-3.404671e-01,7.341397e-01,-3.063432e-01,6.234355e-01
75%,8.192350e-01,5.216658e-01,4.324099e-01,-4.634007e-01,-3.092065e-02,7.341397e-01,-3.063432e-01,6.234355e-01
max,8.192350e-01,3.819653e+00,6.837481e+00,6.858330e+00,9.264254e+00,7.341397e-01,3.264313e+00,6.234355e-01


Notice that mean = 0 and std = 1 (when rounded)

In [98]:
X_test = sc.transform(X_test) # uses the scaling factors used in X_train above. So only transform must be done. No Fitting

In [99]:
type(X_train)

pandas.core.frame.DataFrame

# Now lets make ANN

#### For Jupyter Notebook, make sure Keras and TensorFlow are installed first
- [Keras Installation Instructions](https://keras.io/#installation)  
- [Tensorflow Installation Instructions](https://www.tensorflow.org/install)

In [100]:
# Importing the Keras libraries and packages
import keras
from keras.models import Sequential
from keras.layers import Dense

In [255]:
# Initializing the ANN
classifier = Sequential()

In [256]:
X_train.shape

(711, 8)

In [257]:
# Adding the input layer and the first hidden layer
classifier.add(Dense(units = 9, kernel_initializer = 'uniform', 
                     activation = 'relu', input_dim = X_train.shape[1]))

**ACTIVATION FUNCTIONS**

- relu = filters out negative values (Usually a very effective activation function)
- sigmoid = between 0 and 1.  Provides the probability of survival for titanic passengers
- accuracy = ??

In [258]:
# Adding the second hidden layer
classifier.add(Dense(units = 10, kernel_initializer = 'uniform', activation = 'relu'))

In [259]:
# Adding the third hidden layer
classifier.add(Dense(units = 8, kernel_initializer = 'uniform', activation = 'relu'))

In [260]:
# Adding the output layer
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

In [261]:
# Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [262]:
classifier.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_25 (Dense)             (None, 9)                 81        
_________________________________________________________________
dense_26 (Dense)             (None, 10)                100       
_________________________________________________________________
dense_27 (Dense)             (None, 8)                 88        
_________________________________________________________________
dense_28 (Dense)             (None, 1)                 9         
Total params: 278
Trainable params: 278
Non-trainable params: 0
_________________________________________________________________


In [263]:
# Fitting the ANN to the Training set
classifier.fit(X_train, y_train, verbose=1, batch_size = 100, epochs = 50)

Epoch 1/50
711/711 [==============================] - 1s 913us/step - loss: 0.6928 - acc: 0.6203
Epoch 2/50
711/711 [==============================] - 0s 27us/step - loss: 0.6917 - acc: 0.6245
Epoch 3/50
711/711 [==============================] - 0s 26us/step - loss: 0.6907 - acc: 0.6245
Epoch 4/50
711/711 [==============================] - 0s 43us/step - loss: 0.6896 - acc: 0.6245
Epoch 5/50
711/711 [==============================] - 0s 26us/step - loss: 0.6883 - acc: 0.6245
Epoch 6/50
711/711 [==============================] - 0s 38us/step - loss: 0.6865 - acc: 0.6245
Epoch 7/50
711/711 [==============================] - 0s 32us/step - loss: 0.6843 - acc: 0.6245
Epoch 8/50
711/711 [==============================] - 0s 26us/step - loss: 0.6814 - acc: 0.6245
Epoch 9/50
711/711 [==============================] - 0s 39us/step - loss: 0.6769 - acc: 0.6245
Epoch 10/50
711/711 [==============================] - 0s 25us/step - loss: 0.6707 - acc: 0.6245
Epoch 11/50
711/711 [=================

In [264]:
classifier.weights

[<tf.Variable 'dense_25/kernel:0' shape=(8, 9) dtype=float32_ref>,
 <tf.Variable 'dense_25/bias:0' shape=(9,) dtype=float32_ref>,
 <tf.Variable 'dense_26/kernel:0' shape=(9, 10) dtype=float32_ref>,
 <tf.Variable 'dense_26/bias:0' shape=(10,) dtype=float32_ref>,
 <tf.Variable 'dense_27/kernel:0' shape=(10, 8) dtype=float32_ref>,
 <tf.Variable 'dense_27/bias:0' shape=(8,) dtype=float32_ref>,
 <tf.Variable 'dense_28/kernel:0' shape=(8, 1) dtype=float32_ref>,
 <tf.Variable 'dense_28/bias:0' shape=(1,) dtype=float32_ref>]

In [265]:
#Part 3 - Making predictions and evaluating the model

In [266]:
# Predicting the Test set results
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.4)

In [267]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[80, 25],
       [24, 49]])

In [268]:
print(accuracy_score(y_test, y_pred) )

0.7247191011235955


In [269]:
print(precision_score(y_test, y_pred) )

0.6621621621621622


In [270]:
print(f1_score(y_test, y_pred) )

0.6666666666666666


In [271]:
y_pred[:,0]

array([ True, False,  True,  True, False, False, False, False,  True,
        True,  True,  True,  True,  True,  True, False,  True,  True,
       False,  True, False,  True, False, False, False, False,  True,
       False, False, False,  True, False, False,  True, False, False,
       False, False, False, False,  True, False,  True, False,  True,
       False, False, False, False, False,  True, False, False,  True,
       False,  True,  True,  True, False, False,  True, False,  True,
       False, False, False,  True,  True,  True,  True,  True, False,
       False,  True,  True, False, False, False,  True, False, False,
        True,  True,  True,  True,  True, False, False, False, False,
       False,  True,  True, False, False,  True,  True,  True, False,
        True, False,  True, False,  True, False,  True, False, False,
       False,  True,  True,  True, False,  True, False, False, False,
       False, False, False,  True, False, False,  True, False, False,
       False, False,

# Step - 7 : Predict on New Cases
## Prediction on Test Data From Kaggle

create an account on www.kaggle.com

TODO - LOAD ACTUAL TEST DATA FROM KAGGLE, CLEAN DATA, RUN NEURAL NET MODEL ON DATA, SUBMIT RESULTS

In [272]:
# FOR GOOGLE COLAB - UNCOMMENT LINES BELOW

# !wget -q https://www.dropbox.com/s/t9i5j6ki0qsf989/Titanic%20Data-Test.csv?dl=0
# !ls -l
# test_data = pd.read_csv('Titanic Data-Test.csv')

In [273]:
# FOR JUPYTER NOTEBOOK - UNCOMMENT LINES BELOW

test_data = pd.read_csv('Titanic Data-Test.csv')

In [274]:
test_data.set_index('PassengerId', inplace=True)
test_data.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,
892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [275]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 418 entries, 892 to 1309
Data columns (total 10 columns):
Pclass      418 non-null int64
Name        418 non-null object
Sex         418 non-null object
Age         332 non-null float64
SibSp       418 non-null int64
Parch       418 non-null int64
Ticket      418 non-null object
Fare        417 non-null float64
Cabin       91 non-null object
Embarked    418 non-null object
dtypes: float64(2), int64(3), object(5)
memory usage: 35.9+ KB


In [276]:
test_data.drop(['Name', 'Ticket', 'Cabin'], axis=1, inplace=True)
test_data.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
PassengerId,,,,,,,
892,3,male,34.5,0,0,7.8292,Q
893,3,female,47.0,1,0,7.0000,S
894,2,male,62.0,0,0,9.6875,Q
895,3,male,27.0,0,0,8.6625,S
896,3,female,22.0,1,1,12.2875,S


In [277]:
test_data['Embarked'].value_counts()

S    270
C    102
Q     46
Name: Embarked, dtype: int64

In [278]:
embarked_dummies = pd.get_dummies(test_data['Embarked'], drop_first=True)
embarked_dummies.head()

,Q,S
PassengerId,,
892,1,0
893,0,1
894,1,0
895,0,1
896,0,1


In [279]:
test_data['Sex'].value_counts()

male      266
female    152
Name: Sex, dtype: int64

In [280]:
male = pd.get_dummies(test_data['Sex'], drop_first=True)
male.head()

,male
PassengerId,
892,1
893,0
894,1
895,1
896,0


In [281]:
test_data = pd.concat([test_data, embarked_dummies, male], axis=1)
test_data.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Q,S,male
PassengerId,,,,,,,,,,
892,3,male,34.5,0,0,7.8292,Q,1,0,1
893,3,female,47.0,1,0,7.0000,S,0,1,0
894,2,male,62.0,0,0,9.6875,Q,1,0,1
895,3,male,27.0,0,0,8.6625,S,0,1,1
896,3,female,22.0,1,1,12.2875,S,0,1,0


In [282]:
test_data.drop(['Sex', 'Embarked'], axis=1, inplace=True)
test_data.head()

,Pclass,Age,SibSp,Parch,Fare,Q,S,male
PassengerId,,,,,,,,
892,3,34.5,0,0,7.8292,1,0,1
893,3,47.0,1,0,7.0000,0,1,0
894,2,62.0,0,0,9.6875,1,0,1
895,3,27.0,0,0,8.6625,0,1,1
896,3,22.0,1,1,12.2875,0,1,0


In [283]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 418 entries, 892 to 1309
Data columns (total 8 columns):
Pclass    418 non-null int64
Age       332 non-null float64
SibSp     418 non-null int64
Parch     418 non-null int64
Fare      417 non-null float64
Q         418 non-null uint8
S         418 non-null uint8
male      418 non-null uint8
dtypes: float64(2), int64(3), uint8(3)
memory usage: 20.8 KB


In [284]:
test_data.isnull().sum()

Pclass     0
Age       86
SibSp      0
Parch      0
Fare       1
Q          0
S          0
male       0
dtype: int64

In [285]:
t_mean_age_Pclass1 = int(test_data.loc[test_data['Pclass']==1]['Age'].mean())
t_mean_age_Pclass2 = int(test_data.loc[test_data['Pclass']==2]['Age'].mean())
t_mean_age_Pclass3 = int(test_data.loc[test_data['Pclass']==3]['Age'].mean())

test_data.loc[(test_data['Age'].isnull() ) & (test_data['Pclass']==1), 'Age'] = t_mean_age_Pclass1
test_data.loc[(test_data['Age'].isnull() ) & (test_data['Pclass']==2), 'Age'] = t_mean_age_Pclass2
test_data.loc[(test_data['Age'].isnull() ) & (test_data['Pclass']==3), 'Age'] = t_mean_age_Pclass3

In [286]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 418 entries, 892 to 1309
Data columns (total 8 columns):
Pclass    418 non-null int64
Age       418 non-null float64
SibSp     418 non-null int64
Parch     418 non-null int64
Fare      417 non-null float64
Q         418 non-null uint8
S         418 non-null uint8
male      418 non-null uint8
dtypes: float64(2), int64(3), uint8(3)
memory usage: 20.8 KB


In [287]:
test_data['Fare'].fillna(value=test_data['Fare'].mean(), inplace=True)
test_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 418 entries, 892 to 1309
Data columns (total 8 columns):
Pclass    418 non-null int64
Age       418 non-null float64
SibSp     418 non-null int64
Parch     418 non-null int64
Fare      418 non-null float64
Q         418 non-null uint8
S         418 non-null uint8
male      418 non-null uint8
dtypes: float64(2), int64(3), uint8(3)
memory usage: 20.8 KB


In [288]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
sc.fit(test_data)
X_test = sc.transform(test_data)
X_test = pd.DataFrame(X_test)
X_test.describe()

,0,1,2,3,4,5,6,7
count,4.180000e+02,4.180000e+02,4.180000e+02,4.180000e+02,4.180000e+02,4.180000e+02,4.180000e+02,4.180000e+02
mean,-2.018587e-17,3.027881e-17,-1.593622e-17,-1.407699e-16,-2.808758e-17,-1.407699e-17,-1.593622e-16,1.859225e-17
std,1.001198e+00,1.001198e+00,1.001198e+00,1.001198e+00,1.001198e+00,1.001198e+00,1.001198e+00,1.001198e+00
min,-1.505120e+00,-2.257748e+00,-4.994700e-01,-4.002477e-01,-6.387815e-01,-3.516474e-01,-1.350676e+00,-1.322876e+00
25%,-1.505120e+00,-4.927954e-01,-4.994700e-01,-4.002477e-01,-4.972129e-01,-3.516474e-01,-1.350676e+00,-1.322876e+00
50%,8.734819e-01,-3.381784e-01,-4.994700e-01,-4.002477e-01,-3.796234e-01,-3.516474e-01,7.403703e-01,7.559289e-01
75%,8.734819e-01,5.412057e-01,6.169924e-01,-4.002477e-01,-7.399887e-02,-3.516474e-01,7.403703e-01,7.559289e-01
max,8.734819e-01,3.604555e+00,8.432229e+00,8.781044e+00,8.547081e+00,2.843757e+00,7.403703e-01,7.559289e-01


In [289]:
# Predicting the Test set results
test_y_pred = classifier.predict(X_test)
test_y_pred = (test_y_pred > 0.3)
test_y_pred = pd.DataFrame(test_y_pred)
test_y_pred.tail()

,0
413,False
414,True
415,False
416,False
417,False


In [290]:
results = pd.DataFrame()
results['PassengerId'] = test_data.index
results['Survived'] = test_y_pred
results.tail()

,PassengerId,Survived
413,1305,False
414,1306,True
415,1307,False
416,1308,False
417,1309,False


In [291]:
results['Survived'] = results['Survived'].apply(lambda x: x*1)
results.tail()

,PassengerId,Survived
413,1305,0
414,1306,1
415,1307,0
416,1308,0
417,1309,0


In [292]:
# FOR GOOGLE COLAB - UNCOMMENT LINES BELOW

# filename = "Titanic result ANN 2019-05-04.csv"
# results.to_csv(filename, index=False)
# from google.colab import files
# files.download(filename) 

In [294]:
# FOR JUPYTER NOTEBOOK - UNCOMMENT LINES BELOW AND UPDATE FILENAME

filename = "Titanic Results/Titanic result ANN 2019-05-06.csv"
results.to_csv(filename, index=False)